In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import librosa
import IPython.display as ipd
import copy

import nn_fac.multilayer_nmf as mlnmf
import nn_fac.deep_nmf as dnmf
#from nn_fac.utils.current_plot import *

import base_audio.audio_helper as audio_helper
import base_audio.signal_to_spectrogram as signal_to_spectrogram
import base_audio.spectrogram_to_signal as spectrogram_to_signal

import scripts.encapsulating_scripts as encapsulating_scripts

import tasks.msa as msa_task
import tasks.msa.factorisation_to_signal as factorisation_to_signal
import tasks.msa.plotting_utils as plotting_utils

In [ ]:
# %% Audio params
sr = 44100
hop_length =32
feature_object = signal_to_spectrogram.FeatureObject(sr=sr, feature="stft_complex", hop_length=hop_length, n_fft = 2048)

# %% General params
eps = 1e-12
plotting = False # If you want data to be plotted

# %% Deep NMF params
all_ranks = [32,16,10]
n_iter = 800
n_iter_init_deep_nmf = 100
n_iter_deep = n_iter - n_iter_init_deep_nmf # 100 iterations for the initialization using multi-layer NMF

# Barwise parameters
subdivision = 96
subset_nb_bars = 12
nb_patterns_to_listen_to = 4

What data preprocessing is made here?

This notebook presents Multi and Deep NMF computed on what we call a **Barwise TF matrix**. This representation is not standard, hence it is described hereafter.

Let's start with a spectrogram $X \in \mathbb{R}^{F \times T}$, with $T$ the number of time samples in the song, and $F$ the frequency dimension. It is the standard representation of an audio signal, presented hereafter.

In [ ]:
audio_path = 'data/Come_Together.wav'
signal, _ = librosa.load(audio_path, sr=sr, mono=True)
#plot_me_this_spectrogram(features.get_spectrogram(signal, sr, feature="stft", hop_length=512), title='Spectrogram of the input signal', x_axis='Time T (s)', y_axis='Frequency F (Hz)')

We will split this spectrogram along downbeats, i.e. the first beat of each bar, as presented on a music sheet in the following figure:

<img src="./Concept_bars.png">

This returns a 3-dimensional tensor, coind "TFB-tensor" fot Time-Frequency-Bar tensor. In practice, Time is not represented by the parameter $T$ anymore, but by a parameter $S$,n meaning "subdivision". Indeed, as bars may be of different sizes (for musical reasons), we volontarily oversample the original spectrogram, yielding a large $T$, and then select a fixed number $S$ of samples per bar. Hence, the spectrogram $X \in \mathbb{R}^{F \times T}$ is transformed into a tensor $\mathscr{X} \in \mathbb{R}^{B \times F \times S}$. This tensor is finally unfolded along the first mode, yielding a "Barwise TF matrix" $\in \mathbb{R}^{B \times FS}$. The whole process is presented in the following figure:
<img src="./barwiseTF_creation.png">

Multi and Deep NMF will be computed on this matrix, where each row represents a barwise Time-Frequency vector. Hopefully, if bars are really close, we will catch the most repetitive parts of bars, which wze call "patterns" as they are repeated throughout the song.

This is made in the following code:

In [ ]:
try:
    bars = np.load('data/pre_computed_stuff/come_together_bars.npy', allow_pickle=True)
except FileNotFoundError:
    bars = msa_task.data_manipulation.get_bars_from_audio(audio_path) # Computing the bars
    np.save('data/pre_computed_stuff/come_together_bars.npy', bars)


# As bars may be of different lengths, we compute the median hop length in the song for reconstruction.
median_hop = msa_task.data_manipulation.get_median_hop(bars, subdivision = subdivision, sampling_rate = sr)

# Creating an object to reconstruct signals from spectrogram, using the median hop as default hop length.
reconstruction_feature_object = copy.deepcopy(feature_object)
reconstruction_feature_object.hop_length = median_hop
reconstruction_feature_object.feature = "stft"

oversampled_spectrogram_complex = feature_object.get_spectrogram(signal)
# Barwise TF matrix (magnitude and phase)
original_barwise_tf_matrix_mag = msa_task.barwise_input.barwise_TF_matrix(oversampled_spectrogram_complex[0], bars, feature_object.hop_length/sr, subdivision) + eps
original_barwise_tf_matrix_phase = msa_task.barwise_input.barwise_TF_matrix(oversampled_spectrogram_complex[1], bars, feature_object.hop_length/sr, subdivision)

original_signal = factorisation_to_signal.TF_matrix_to_audio_signal(original_barwise_tf_matrix_mag, reconstruction_feature_object, subdivision=subdivision, 
                                                                    subset_nb_bars = None, phase_retrieval = "original_phase", barwise_tf_original_phase=original_barwise_tf_matrix_phase)

# original_signal_subset_bars = factorisation_to_signal.TF_matrix_to_audio_signal(original_barwise_tf_matrix_mag, feature="stft", frequency_dimension=1025, subdivision=subdivision, 
#                                                                                 subset_nb_bars = subset_nb_bars, phase_retrieval = "original_phase", hop_length = median_hop, barwise_tf_original_phase=original_barwise_tf_matrix_phase, sr=44100)

# original_phase_subset_bars = model.barwise_input.TF_matrix_to_spectrogram(original_barwise_tf_matrix_phase,frequency_dimension=1025, subdivision=subdivision,subset_nb_bars=subset_nb_bars)
# original_mag_subset_bars = model.barwise_input.TF_matrix_to_spectrogram(original_barwise_tf_matrix_mag,frequency_dimension=1025, subdivision=subdivision,subset_nb_bars=subset_nb_bars)


All the aformentioned process talks about spectrogram, but dealing with audio signals offer the possibility to listen to our data. In that spirit, we want to be able to transform a real-valued spectrogram into a complex spectrogram, then transformed into an audio signal via the Inverse Fourier Transform. The following code proposes two ways to reconstruct an audio signal (i.e. estimating the phase iformation) from a real-valued spectrogram:
- Using the Griffin_Lim algorithm [REF TO ADD], a classical algorithm in phase reconstruction,
- Using the phase of the original signal as an approximation of the phase contained in the spectrogram. This makes sense because we are aiming at reconstructing as closely as possible the original spectrogram. Hence, if the optimization scheme worked, the reconstructed spectrogram should be close to the original one, and the phase information should hence also be close.

Both are presented on the original Fourier transform of the song.

In [ ]:
print("One particular bar")
bar=4
TF_vector_magnitude_this_bar = original_barwise_tf_matrix_mag[bar]
TF_vector_phase_this_bar = original_barwise_tf_matrix_phase[bar]
signal = factorisation_to_signal.TF_vector_to_audio_signal(TF_vector_magnitude_this_bar, reconstruction_feature_object, subdivision=subdivision, phase_retrieval = "original_phase", tf_vector_original_phase=TF_vector_phase_this_bar)
audio_helper.listen_to_this_signal(signal, sr=sr)

Finally, the original song is Come Together, by The Beatles. It's structure in bars is displayed hereafter, and this is what we aim at finding in the $W$ matrix:

In [ ]:
Q = np.zeros([89,7])
intro = [0, 1, 2, 3, 12,13,14,15, 26,27,28,29, 40,41, 50,51, 62,63,64,65]
verse_a = [4,5,6,7, 16,17,18,19, 30,31,32,33, 52,53,54,55]
verse_b = [8,9,10,11, 20,21,22,23, 34,35,36,37, 56,57,58,59]
chorus = [24,25, 38,39, 60,61]
solo_1 = [42,43,44,45]
solo_2 = [46,47,48,49]
outro = [66,67,68,69, 70,71,72,73, 74,75,76,77, 78,79,80,81, 82,83,84,85, 86,87,88]
Q[intro, 0] = 1
Q[verse_a, 1] = 1
Q[verse_b, 2] = 1
Q[chorus, 3] = 1
Q[solo_1, 4] = 1
Q[solo_2, 5] = 1
Q[outro, 6] = 1
plotting_utils.plot_permuted_factor(Q, title = 'Structure annotation for Come Together', x_axis='bars', y_axis='sections')

In [ ]:
# For the tests hereafter, define how to retrieve the phase
phase_retrieval = "original_phase" # Original phase returns cleaner audio signals

In [ ]:
W_multi, H_multi, errors_multi, toc_multi = mlnmf.multilayer_beta_NMF(original_barwise_tf_matrix_mag, all_ranks = all_ranks, beta = 1, n_iter_max_each_nmf = n_iter, return_errors = True)

We show the $W$ matrix in the last layer, which, hopefully, resembles the structure annotation matrix

In [ ]:
plotting_utils.plot_permuted_factor(W_multi[-1], "W matrix for bottom layer, to be compared with the structure annotation figure.", x_axis='bars', y_axis='patterns (rows in H)')

We also listen to the reconstructed song, depending on the level of the decomposition.

In [ ]:
encapsulating_scripts.evaluate_multi_nmf(W_multi, H_multi, reconstruction_feature_object=reconstruction_feature_object, barwise_TF_original_mag=original_barwise_tf_matrix_mag, barwise_TF_original_phase=original_barwise_tf_matrix_phase, phase_retrieval=phase_retrieval, subset_nb_bars=subset_nb_bars, subdivision=subdivision)

In [ ]:
W_deep, H_deep, errors_deep, toc_deep = dnmf.deep_KL_NMF(original_barwise_tf_matrix_mag, all_ranks = all_ranks, n_iter_max_each_nmf = n_iter_init_deep_nmf, n_iter_max_deep_loop = n_iter_deep,return_errors=True)


In [ ]:
# Evolution of the erros at the different levels of 
# deep β-NMF with β = 1 (initialized with multilayer β-NMF after 250 iterations) 
# divided by the error of multilayer β-NMF after 500 iterations.
plt.figure(1)
plt.plot(errors_deep[0,1:], color='blue', label='Layer 1')
plt.plot(errors_deep[1,1:], color='red', label='Layer 2')
plt.plot(errors_deep[2,1:], color='black', label='Layer 3')
plt.xlabel('Iterations')
plt.ylabel('Ratio deep vs. multilayer')
# plt.title(r'Computation $x^\star$')
plt.legend()
plt.show()

We show the $W$ matrix in the last layer, which, hopefully, resembles the structure annotation matrix

In [ ]:
# Check the constraints
l = 0
print("Layer 1:", np.sum(H_deep[l], axis=1))
l += 1
print("Layer 2:", np.sum(H_deep[l], axis=1))
l += 1
print("Layer 2:", np.sum(H_deep[l], axis=1))
      

In [ ]:
plotting_utils.plot_permuted_factor(W_deep[0], "Q matrix of the top layer.", x_axis='bars', y_axis='Patterns (rows in H)')
plotting_utils.plot_permuted_factor(W_deep[1], "Q matrix of the intemediate layer.", x_axis='bars', y_axis='Patterns (rows in H)')
plotting_utils.plot_permuted_factor(W_deep[2], "Q matrix of the bottom layer.", x_axis='bars', y_axis='Patterns (rows in H)')

We also listen to the reconstructed song, depending on the level of the decomposition.

In [ ]:
encapsulating_scripts.evaluate_multi_nmf(W_deep, H_deep, reconstruction_feature_object=reconstruction_feature_object, barwise_TF_original_mag=original_barwise_tf_matrix_mag, barwise_TF_original_phase=original_barwise_tf_matrix_phase, phase_retrieval=phase_retrieval, subset_nb_bars=subset_nb_bars, subdivision=subdivision)

In [ ]:
plotting_utils.plot_me_this_spectrogram(W_deep[0].T)#, "Q matrix of the top layer.", x_axis='bars', y_axis='Patterns (rows in H)')
plotting_utils.plot_me_this_spectrogram(W_deep[1].T)#, "Q matrix of the intemediate layer.", x_axis='bars', y_axis='Patterns (rows in H)')
plotting_utils.plot_me_this_spectrogram(W_deep[2].T)#, "Q matrix of the bottom layer.", x_axis='bars', y_axis='Patterns (rows in H)')